In [42]:
# Setup for Google Colab directories
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
%cd MLexercise_CarDimensions

# import glob
# import re
import pandas as pd
# file_paths = glob.glob("data/2*_en.csv")
dataframes = pd.read_csv("data/cleaned_data.csv")
print(dataframes.columns.tolist())
dataframes

/content/drive/MyDrive/Colab Notebooks/MLexercise_CarDimensions
['MAKE', 'MODEL', 'Model Year', 'Overall Length', 'Overall Width', 'Overall Height', 'Wheelbase', 'Curb Weight', 'Front End Length', 'Rear End Length', 'Side Glass Height', 'Body Side Height', 'Roof Width', 'Front Overhang', 'Rear Overhang', 'Track Width Front', 'Track Width Rear', 'Weight Distribution', 'Front Weight %', 'Rear Weight %']


,MAKE,MODEL,Model Year,Overall Length,Overall Width,Overall Height,Wheelbase,Curb Weight,Front End Length,Rear End Length,Side Glass Height,Body Side Height,Roof Width,Front Overhang,Rear Overhang,Track Width Front,Track Width Rear,Weight Distribution,Front Weight %,Rear Weight %
0,FIAT,124SPIDER2DRCONVERTIBLE,16,407.0,173.0,124.0,231.0,1114.0,155.0,79.0,31.0,67.0,116.0,85.0,91.0,152.0,150.0,54/46,54.0,46.0
1,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,208.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
2,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,202.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
3,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,208.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
4,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,202.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,BMW,Z4SDRIVE35I35IS2DRCONVERTIBLERWD,9,424.0,179.0,129.0,250.0,1610.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3760,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1585.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3761,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1590.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3762,ACURA,ZDX4DRSUVAWDTECHELITE,10,488.0,199.0,160.0,275.0,2064.0,133.0,109.0,33.0,88.0,120.0,109.0,104.0,172.0,171.0,58/42,58.0,42.0


In [43]:
# # Features and target variable
features = [
'Overall Length', 'Overall Width', 'Overall Height',
'Front End Length', 'Rear End Length',
'Side Glass Height', 'Body Side Height',
'Wheelbase', 'Front Overhang', 'Rear Overhang', # Sum = Overall Length
'Roof Width', 'Track Width Front', 'Track Width Rear', # ~ Overall Width
]

target = 'Curb Weight'

X = dataframes[features]
y = dataframes[target]


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

In [44]:
# Model layers and training
model = Sequential([
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # output layer for regression
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 3458219.2500 - val_loss: 3366143.7500
Epoch 2/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3339778.5000 - val_loss: 2966576.7500
Epoch 3/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2721420.2500 - val_loss: 2044532.5000
Epoch 4/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1754571.3750 - val_loss: 1218294.2500
Epoch 5/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1095437.6250 - val_loss: 905655.2500
Epoch 6/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 795544.7500 - val_loss: 753726.4375
Epoch 7/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 673865.9375 - val_loss: 647922.1875
Epoch 8/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 577258.6250 - val_loss: 570650.8125
Epoch 9/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 530155.1250 - val_loss: 503232.0000
Epoch 10/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 453571.1562 - val_loss: 442614.2500
Epoch 11/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 

In [45]:
y_pred = model.predict(X_test)
# print(y_pred)

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [47]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

print("R² Score:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("Mean target weight:", y.mean())
print("Mean target predicted:", y_pred.mean())
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape * 100, "%")


R² Score: 0.8166923045170863
MSE: 27972.214207481044
Mean target weight: 1808.4378320935175
Mean target predicted: 1814.8937
MAPE: 6.499983295855337 %


In [48]:
from notebooks.utils import compare_predictions

comparison = compare_predictions(y_test, y_pred)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(comparison.head(100))

    Actual    Predicted       Error    Error %
0   1210.0  1174.897461  -35.102539   2.901036
1   2606.0  2496.397217 -109.602783   4.205786
2   1365.0  1389.314575   24.314575   1.781288
3   1818.0  1860.116089   42.116089   2.316617
4   1832.0  1833.511841    1.511841   0.082524
5   2137.0  2040.563965  -96.436035   4.512683
6   1717.0  1944.781494  227.781494  13.266249
7   2123.0  2238.130127  115.130127   5.422992
8   1934.0  1913.434937  -20.565063   1.063344
9   2096.0  2210.283691  114.283691   5.452466
10  1389.0  1550.835327  161.835327  11.651211
11  2085.0  2370.436279  285.436279  13.689989
12  2052.0  2019.804443  -32.195557   1.568984
13  2047.0  1947.372803  -99.627197   4.866986
14  1552.0  1641.756714   89.756714   5.783293
15  1495.0  1625.088989  130.088989   8.701605
16  1930.0  1884.469849  -45.530151   2.359075
17  1575.0  1463.247681 -111.752319   7.095385
18  1420.0  1633.848511  213.848511  15.059754
19  1586.0  1474.115112 -111.884888   7.054533
20  2010.0  1

In [49]:
y_pred_full = model.predict(X)

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [50]:
full_comparison = compare_predictions(y, y_pred_full)
full_comparison.to_csv("results/weight_prediction_TF.csv", index=False)